In [5]:
import os
import time
import pandas as pd
import boto3
import json
from dotenv import load_dotenv
from sklearn.metrics import accuracy_score
import re

def es_ticket_basura(texto):
    """Detecta tickets que no tienen suficiente contexto para ser procesados."""
    palabras = texto.strip().split()
    # Si tiene 2 palabras o menos, y son palabras genéricas, lo descartamos
    basura_comun = ['consulta', 'solicitud', 'ayuda', 'urgente', 'problema', 'error', 'ticket']
    
    if len(palabras) <= 2:
        if all(p.lower() in basura_comun for p in palabras):
            return True
    return False

categorias_validas = ['ABM',
 'Soporte de Campo',
 'Aplicaciones',
 'Ingenieros TI',
 'Folios OT',
 'Sin Categoria',
 'DBA',
 'MDA',
 'Técnico TI',
 'TDO',
 'OT',
 'SAP',
 'Sistemas GIS',
 'Ciberseguridad',
 'Reinicio Servidor',
 'Soporte EO',
 'eTerra',
 'FTP/SFTP',
 'Telecomunicaciones',
 'DTE',
 'Meridian',
 'Incidente',
 'NARI',
 'Soporte APM',
 'PowerOn',
 'Monitoreo',
 'Redirección Correo',
 'OSF - Archivos OSF',
 'Tecnored - Sistemas',
 'Telecomunicaciones Operacionales',
 'AMI',
 'Seguimientos IMA',
 'Actividades Proyectos IMA',
 'Desarrollo de Sistemas y Proyectos Tecnológicos',
 'Análisis Nessus',
 'Servicio Docker']

# --- 1. CARGA DE CREDENCIALES AWS ---
load_dotenv() 

entorno = os.getenv("ENV", "dev")
aws_region = os.getenv("AWS_REGION", "us-east-1")
model_id = os.getenv("DEEPSEEK_MODEL_ID") 

# Inicializamos el cliente de AWS Bedrock (Si usas SageMaker, cambia a 'sagemaker-runtime')
try:
    aws_client = boto3.client(
        service_name='bedrock-runtime',  # Cambiar a 'sagemaker-runtime' si usaste JumpStart
        region_name=aws_region,
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
    )
    print(f"✅ Cliente AWS configurado. Ejecutando en modo: {entorno.upper()}")
except Exception as e:
    raise ValueError(f"❌ Error configurando AWS: {e}")

# --- 2. CARGA DE DATOS ---
try:
    df_dudas = pd.read_csv('tickets_para_agente_ia.csv')
except FileNotFoundError:
    print("❌ Archivo no encontrado. Sube tickets_para_agente_ia.csv")

if entorno == "dev":
    print(f"🛠️ MODO DEV: Usando muestra aleatoria de 30 tickets.")
    df_procesar = df_dudas.sample(n=30, random_state=42).copy()
else:
    print(f"🚀 MODO PROD: Evaluando TODOS los tickets del archivo.")
    df_procesar = df_dudas.copy()

# --- 3. PROMPT ENGINEERING PARA DEEPSEEK ---
categorias_validas = categorias_validas

def clasificar_con_deepseek_avanzado(ticket_texto):
    prompt = f"""Eres un Despachador Senior de una Mesa de Ayuda TI en Chile.
    Tu tarea es analizar el título de un ticket y clasificarlo ESTRICTAMENTE en UNA de estas categorías:
    {categorias_validas}

    DICCIONARIO DE LA EMPRESA:
    - DBA: Cuentas caducadas/bloqueadas, OSF, accesos GLPI, base de datos.
    - MDA: Cambio de contraseña, Cloudsigner, menú general de sistemas.
    - Técnico TI: Restablecer contraseña de usuario, accesos Clevest, personal propio.
    - Soporte de Campo: Citrix, Electric Office (EO), configuración física.
    - Aplicaciones: ORM, IVR, Whatsapp, Cucofepa, diccionario de datos.
    - ABM: Creación de usuario, accesos nuevos (Clevest, ORM), adicionar rol.
    - Seguimientos IMA: "seguimiento", "conjunto", "análisis requerimiento", "revisión OT".
    - Desarrollo de Sistemas y Proyectos Tecnológicos: MGS, STAR, autoconsulta.
    - Ciberseguridad: Python, Trellix, Cortex, Phishing, Nessus.

    EJEMPLOS DE CLASIFICACIÓN (Aprende de estos casos):
    Ticket: "es acceder a electric office citrix" -> <categoria>Soporte de Campo</categoria>
    Ticket: "consulta josefa ignacia lohaus" -> <categoria>ABM</categoria>
    Ticket: "solucion operativa ch250963405" -> <categoria>Aplicaciones</categoria>
    Ticket: "seguimiento conjunto temas osf" -> <categoria>Seguimientos IMA</categoria>
    Ticket: "dbaparral de 4 a 2" -> <categoria>DBA</categoria>

    INSTRUCCIONES FINALES:
    Analiza el ticket. Luego, entrega TU RESPUESTA FINAL OBLIGATORIAMENTE dentro de las etiquetas <categoria></categoria>.

    Ticket a clasificar: "{ticket_texto}"
    Respuesta:"""

    try:
        response = aws_client.converse(
            modelId=model_id,
            messages=[{"role": "user", "content": [{"text": prompt}]}],
            inferenceConfig={"temperature": 0.0} # Temperatura 0 para cero creatividad/alucinación
        )
        
        respuesta_cruda = response['output']['message']['content'][0]['text']
        
        # --- EXTRACCIÓN INFALIBLE CON REGEX ---
        # Buscamos exactamente lo que está dentro de <categoria>...</categoria>
        match = re.search(r'<categoria>(.*?)</categoria>', respuesta_cruda, re.IGNORECASE)
        
        if match:
            return match.group(1).strip()
        else:
            # Si DeepSeek olvidó las etiquetas, intentamos nuestro parseo antiguo
            if "</think>" in respuesta_cruda:
                return respuesta_cruda.split("</think>")[-1].strip()
            return respuesta_cruda.strip()

    except Exception as e:
        print(f"Error de API AWS: {e}")
        return "Error_API"

    except Exception as e:
        print(f"Error de API AWS: {e}")
        return "Error_API"

# --- EJECUCIÓN OPTIMIZADA ---
print(f"\n🤖 Iniciando Agente DeepSeek-R1 (V2.0) para {len(df_procesar)} tickets...")
predicciones = []
ahorro_api = 0

for index, row in df_procesar.iterrows():
    texto_ticket = row['Ticket']
    
    # 1. Filtro Anti-Basura (Ahorramos la llamada a AWS)
    if es_ticket_basura(texto_ticket):
        prediccion = "Sin Categoria"
        ahorro_api += 1
        print(f"Ticket: {texto_ticket[:40]:<40} | Pred: {prediccion:<25} | ⚡ Filtro Automático")
    else:
        # 2. Llamada a la IA
        time.sleep(0.5) 
        prediccion = clasificar_con_deepseek_avanzado(texto_ticket)
        print(f"Ticket: {texto_ticket[:40]:<40} | Pred: {prediccion:<25} | Real: {row['Real']}")
        
    predicciones.append(prediccion)

df_procesar['Pred_Agente'] = predicciones

# --- EVALUACIÓN ---
df_validos = df_procesar[df_procesar['Pred_Agente'].isin(categorias_validas)]
errores_formato = len(df_procesar) - len(df_validos)

if not df_validos.empty:
    accuracy_ia = accuracy_score(df_validos['Real'], df_validos['Pred_Agente'])
    print("\n" + "="*60)
    print("🏆 RESULTADOS DEL AGENTE DEEPSEEK-R1 (Versión Perfeccionada)")
    print("="*60)
    print(f"Precisión en la Zona Gris: {accuracy_ia:.2%}")
    print(f"Errores de formato: {errores_formato}")
    print(f"💰 Llamadas a API ahorradas por filtro basura: {ahorro_api} tickets.")
else:
    print("⚠️ Error grave de formato. Revisa el Regex.")

✅ Cliente AWS configurado. Ejecutando en modo: DEV
🛠️ MODO DEV: Usando muestra aleatoria de 30 tickets.

🤖 Iniciando Agente DeepSeek-R1 (V2.0) para 30 tickets...
Ticket: cuenta bloqueada                         | Pred: DBA                       | Real: DBA
Ticket: dbaparral de 4 a 2                       | Pred: DBA                       | Real: DBA
Ticket: rechazo solicitud n100323lote anulado n1 | Pred: Sin Categoria             | Real: Aplicaciones
Ticket: accesos clevest                          | Pred: ABM                       | Real: ABM
Ticket: problemas para entrar a cloudsigner      | Pred: MDA                       | Real: MDA
Ticket: reestablecer contraseña de usuario       | Pred: Técnico TI                | Real: Técnico TI
Ticket: es acceder a electric office citrix      | Pred: Soporte de Campo          | Real: Soporte de Campo
Ticket: consulta                                 | Pred: Sin Categoria             | ⚡ Filtro Automático
Ticket: cambio de contrasea            